In [1]:
import pandas as pd
import numpy as np
import math
from PIL import Image
from pathlib import Path
from datetime import datetime, timedelta

In [2]:
def unique_value(array, property):
    array = array[property]
    array = array.drop_duplicates(keep='last')
    array = array.tolist()
    array.sort()
    return array

routes = pd.read_csv('../covid_dataset/raw/Figure.csv')
dates = unique_value(routes, 'date')
patients = unique_value(routes, 'patient_id')

In [3]:
size = 256
kernel_size = 3
weight = 10

In [4]:
names = ['age', 'sex', 'infection_case', 'type', 'date']
counts = [11, 2, 4, 21, 7]
visit_types = ['karaoke', 'gas_station', 'gym', 'bakery', 'pc_cafe',
              'beauty_salon', 'school', 'church', 'bank', 'cafe',
              'bar', 'post_office', 'real_estate_agency', 'lodging',
              'public_transportation', 'restaurant', 'etc', 'store',
              'hospital', 'pharmacy', 'airport']
causes = ['community infection', 'etc', 'contact with patient', 'overseas inflow']

In [5]:
class Patient:
    def age_category(age):
        age = int(age[:-1])
        if age == 0: return 0
        elif age == 100: return 10
        return age // 10
            
    def sex_category(sex):
        if sex == 'male': return 0
        return 1

    def infection_case_category(infection_case, causes):
        return causes.index(infection_case)
    
    def type_category(visit_type, move_types):
        return move_types.index(visit_type)

    def day_category(day):
        day = datetime.strptime(day, "%Y-%m-%d")
        return day.weekday()

In [6]:
def put_triangular_kernel(array, row, col, value):
    stride = int((kernel_size - 1) / 2)
    ratio = 1 / (stride + 1)
    
    for i in range(row - stride, row + stride + 1):
        if i < 0 or i >= array.shape[0]: continue
        for j in range(col - stride, col + stride + 1):
            if j < 0 or j >= array.shape[1]: continue
            distance = math.sqrt((row - i)**2 + (col - j)**2)
            new_value = value * (1 - (distance * ratio))
            if new_value < 0: new_value = 0
            array[i][j] = new_value
        
    array[row][col] = value
    return array

In [7]:
def overlay_kernel(array):
#     new_image = np.full((array.shape[0],array.shape[1]), 255.0)
    new_image = np.zeros((array.shape[0],array.shape[1]))
    for row in range(array.shape[0]):
        for col in range(array.shape[1]):
            if array[row][col] == 0: continue
            new_image += put_triangular_kernel(np.zeros((array.shape[0], array.shape[1])), row, col, array[row][col])
#     image_array = new_image
    return new_image

# def put_triangular_kernel(array, row, col, value):
#     stride_origin = int((kernel_size - 1) / 2)
#     stride = int((kernel_size + 1) / 2)
#     stride_1 = int((kernel_size + 1) / 2)
#     stride_distance = math.sqrt(2 * (stride_1**2))
    
#     for i in range(row - stride + 1, row + stride):
#         if i < 0 or i >= array.shape[0]: continue
#         for j in range(col - stride +1, col + stride):
#             if j < 0 or j >= array.shape[1]: continue
#             distance = math.sqrt((row - i)**2 + (col - j)**2)
#             new_value = -1 * (distance / stride_distance) + value
#             array[i][j] = new_value
            
#     array[row][col] = value
    
#     return array

def indices_save_image(path, place_indices):
    all_counts = sum(count for count in counts)
    visit_grid = np.zeros((all_counts, size, size))
    
    for index in place_indices:
        row = index[5]
        col = index[6]
        for feature in range(5):
            visit_grid[index[feature]][row][col] += weight
    
    for channel in range(visit_grid.shape[0]):
        save_grid(path + str(channel) + ".png", visit_grid[channel])
    
def save_grid(path, grid):
    grid = overlay_kernel(grid)
    img = Image.fromarray(grid.astype('uint8'), 'L')
    img.save(path)
    
def save_patient_route(path, patient, routes):
    patient_places = routes[routes['patient_id']==patient]
    patient_dates = unique_value(patient_places, 'date')
    first_day = datetime.strptime(patient_dates[0], "%Y-%m-%d")
    last_day = datetime.strptime(patient_dates[-1], "%Y-%m-%d") # + timedelta(days=3)
    delta = last_day - first_day
    duration = delta.days + 1

    # 저장
    patient_path = path + str(patient)
    Path(patient_path).mkdir(parents=True, exist_ok=True)

    today = first_day
    while(True):
        today_str = datetime.strftime(today, "%Y-%m-%d")
        patient_day_places = patient_places[patient_places['date']==today_str]
        places_indices = combine_places(patient_day_places, counts, causes, visit_types)
        patient_date_path = patient_path + "/" + today_str + '/'
        Path(patient_date_path).mkdir(parents=True, exist_ok=True)
        indices_save_image(patient_date_path, places_indices)
        if today == last_day: break
        today += timedelta(days=1)
        
def combine_places(places, counts, causes, visit_types):
    indices = []
    for i in range(len(places)):
        one_visit = places.iloc[i]
        indices.append(df_to_grid_index(one_visit, counts, causes, visit_types))
    return indices

def df_to_grid_index(one_visit, counts, causes, visit_types):
    index = 0
    p_age = Patient.age_category(one_visit['age'])
    index += counts[0]
    p_sex = Patient.sex_category(one_visit['sex']) + index
    index += counts[1]
    p_infection_case = Patient.infection_case_category(one_visit['infection_case'], causes) + index
    index += counts[2]
    p_type = Patient.type_category(one_visit['type'], visit_types) + index
    index += counts[3]
    p_date = Patient.day_category(one_visit['date']) + index
    row = one_visit['row']
    col = one_visit['col']
    
    return [p_age, p_sex, p_infection_case, p_type, p_date, row, col]

path = '../patient_test/'
for patient in patients:
    save_patient_route(path, patient, routes)

In [8]:
def accumulate_two_days(day1, day2):
    day2[1].extend(day1[1])
    return day2
    
def accumulate_patient(patient, routes):
    patient_route = get_patient_route(patient, routes)
    patient_days = len(patient_route)

    second = patient_days - 1
    first = second - 1
    for i in range(2 * patient_days - 3):
        patient_route[second] = accumulate_two_days(patient_route[first], patient_route[second])
        if second - first == 2: second -=1
        else: first -= 1
    
    return patient_route

def save_patient_routes(path, patient, patient_routes):
    path += str(patient) + '/'
    Path(path).mkdir(parents=True, exist_ok=True)
    for routes in patient_routes:
        patient_date_path = path + routes[0] + '/'
        Path(patient_date_path).mkdir(parents=True, exist_ok=True)
        indices_save_image(patient_date_path, routes[1])
        
def get_patient_route(patient, routes):
    patient_places = routes[routes['patient_id']==patient]
    patient_dates = unique_value(patient_places, 'date')
    first_day = datetime.strptime(patient_dates[0], "%Y-%m-%d")
    last_day = datetime.strptime(patient_dates[-1], "%Y-%m-%d") # + timedelta(days=3)
    delta = last_day - first_day
    duration = delta.days + 1

    patient_routes = []
    today = first_day
    while(True):
        today_str = datetime.strftime(today, "%Y-%m-%d")
        patient_day_places = patient_places[patient_places['date']==today_str]
        places_indices = combine_places(patient_day_places, counts, causes, visit_types)
        patient_routes.append([today_str, places_indices]) 

        if today == last_day: break
        today += timedelta(days=1)
        
    return patient_routes

path = '../accumulated_test/'
for patient in patients:
    accumulated_routes = accumulate_patient(patient, routes)
    save_patient_routes(path, patient, accumulated_routes)

In [9]:
def get_complete_routes(routes, dates, patients):
    first_day = datetime.strptime(dates[0], "%Y-%m-%d")
    last_day = datetime.strptime(dates[-1], "%Y-%m-%d") # + timedelta(days=3)

    # 날짜별 경로 배열 생성
    today = first_day
    complete_routes = []
    while(True):
        today_str = datetime.strftime(today, "%Y-%m-%d")
        places = []
        complete_routes.append([today_str, places])

        if today == last_day: break
        today += timedelta(days=1)

    # 환자 경로 가져온 다음 날짜대로 배치
    for patient in patients:
        accumulated_routes = accumulate_patient(patient, routes)
        for each_route in accumulated_routes:
            route_day = datetime.strptime(each_route[0], "%Y-%m-%d")
            route_places = each_route[1]
            index = (route_day - first_day).days
            complete_routes[index][1].extend(route_places)
    return complete_routes

In [11]:
complete_routes = get_complete_routes(routes, dates, patients)

path = '../complete_test/'
Path(path).mkdir(parents=True, exist_ok=True)


aaa = complete_routes[10][1]    
print(aaa)

visit_grid = np.zeros((256, 256))

for index in aaa:
    row = index[5]
    col = index[6]
    for feature in range(5):
        visit_grid[row][col] += weight

kernel_size = 120
aaa = np.asarray(aaa)
aaa = overlay_kernel(aaa)
    
save_grid("../aaa.png", visit_grid)

[[3, 11, 13, 33, 44, 170, 100], [3, 11, 13, 33, 44, 229, 65], [2, 12, 15, 33, 44, 139, 178], [2, 12, 15, 33, 44, 139, 178], [2, 12, 15, 33, 43, 139, 178], [2, 12, 15, 33, 42, 139, 178], [2, 12, 15, 33, 42, 139, 178]]


In [10]:
import numpy as np
from itertools import chain
from numpy import maximum, minimum

a = [[0, 0, 0, 0, 0, 0, 0],
    [0, 10, 0, 0, 10, 0, 0],
    [0, 10, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 10, 0],
    [0, 0, 0, 0, 0, 0, 0]]

b = [[0, 70, 0, 0, 0, 0, 0],
    [0, 70, 0, 0, 70, 0, 0],
    [0, 0, 70, 0, 0, 0, 0],
    [0, 0, 0, 70, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 70, 0],
    [0, 0, 0, 0, 0, 0, 0]]

c = [[0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0]]

a = np.asarray(a)
b = np.asarray(b)
c = np.asarray(c)

In [15]:
from math import sqrt
from sklearn.metrics import mean_squared_error

abc = sqrt(mean_squared_error([4], [2]))
abc

2.0

In [11]:
result_max = maximum(a, b)
result_min = minimum(a, b)
result_min

array([[ 0,  0,  0,  0,  0,  0,  0],
       [ 0, 10,  0,  0, 10,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0, 10,  0],
       [ 0,  0,  0,  0,  0,  0,  0]])

In [15]:
d = overlay_kernel(a)

for i in range(d.shape[0]):
    for j in range(d.shape[1]):
        print("%.3f" % d[i][j], end=' ')
    print()

9.500 9.646 9.500 9.500 9.646 9.500 0.000 
19.146 19.646 19.146 9.646 10.000 9.646 0.000 
19.146 19.646 19.146 9.500 9.646 9.500 0.000 
9.500 9.646 9.500 0.000 0.000 0.000 0.000 
0.000 0.000 0.000 0.000 9.500 9.646 9.500 
0.000 0.000 0.000 0.000 9.646 10.000 9.646 
0.000 0.000 0.000 0.000 9.500 9.646 9.500 
